In [1]:
from envs.doom_env import DoomEnv

# Initializing environment
env = DoomEnv('vizdoom/scenarios/basic.cfg', True)

In [ ]:
print(env.observation_space.shape)

In [2]:
from agents.ppo_agent import Ppo2Agent

# Creating ppo agent
agent = Ppo2Agent(env)

# Initializing training variables
training_batch_size = 300
number_of_games = 500

num_learning_iterations = 0
best_score = 0
score_history = []

creating actor network...
Metal device set to: Apple M1 Max

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB

successfully created actor network!
creating critic network...
successfully created critic network!
creating optimizer for actor network...
successfully created optimizer for actor network!
creating optimizer for critic network...
successfully created optimizer for critic network!


In [3]:
import numpy as np

for i in range(number_of_games):
    observation = env.reset()
    done = False
    score = 0
    while not done:
        action, probabilities, val = agent.get_optimal_action(observation)
        observation_, reward, done, info = env.step(action)
        score += reward
        agent.remember_experience(observation, action, probabilities, val, reward, done)
        
        if agent.num_of_experiences() >= training_batch_size or done:
            agent.try_learning_from_experience(training_batch_size)
            num_learning_iterations += 1

        observation = observation_

    score_history.append(score)
    avg_score = np.mean(score_history[-100:])

    if avg_score > best_score:
        best_score = avg_score
        agent.save_models()

    print('episode', i, 'score %.1f' % score, 'best_score %.1f' % best_score, 'avg_score %.1f' % avg_score,
          'time_steps', training_batch_size, 'learning_steps', num_learning_iterations)

saving models ...


INFO:tensorflow:Assets written to: models/actor_1683446772038/assets


INFO:tensorflow:Assets written to: models/actor_1683446772038/assets


INFO:tensorflow:Assets written to: models/critic_1683446772710/assets


INFO:tensorflow:Assets written to: models/critic_1683446772710/assets


successfully saved models!
episode 0 score 59.0 best_score 59.0 avg_score 59.0 time_steps 300 learning_steps 1
episode 1 score -350.0 best_score 59.0 avg_score -145.5 time_steps 300 learning_steps 2
episode 2 score -350.0 best_score 59.0 avg_score -213.7 time_steps 300 learning_steps 3
episode 3 score -350.0 best_score 59.0 avg_score -247.8 time_steps 300 learning_steps 4
episode 4 score -226.0 best_score 59.0 avg_score -243.4 time_steps 300 learning_steps 5
episode 5 score -365.0 best_score 59.0 avg_score -263.7 time_steps 300 learning_steps 6
episode 6 score -335.0 best_score 59.0 avg_score -273.9 time_steps 300 learning_steps 7
episode 7 score 95.0 best_score 59.0 avg_score -227.8 time_steps 300 learning_steps 8
episode 8 score -8.0 best_score 59.0 avg_score -203.3 time_steps 300 learning_steps 9
episode 9 score 10.0 best_score 59.0 avg_score -182.0 time_steps 300 learning_steps 10
episode 10 score -350.0 best_score 59.0 avg_score -197.3 time_steps 300 learning_steps 11
episode 11 s

In [ ]:
# Saving models
agent.save_models()

In [4]:
env.close()